# nanoGPT: Глубокое погружение в архитектуру

В этом блокноте мы детально изучим реализацию nanoGPT, загрузим реальные веса и посмотрим на внутреннее устройство слоев.

In [1]:
import torch
import torch.nn as nn
import sys
import os
import inspect
import matplotlib.pyplot as plt

from src.model import GPTLanguageModel, Block, MultiHeadAttention, Head, FeedFoward

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Устройство: {device}")

Using device: mps
Using device: mps
Устройство: mps


## 1. Загрузка модели и весов

Загрузим модель и чекпоинт `model_ckpt.pt`.

In [2]:
model = GPTLanguageModel().to(device)

# Загружаем чекпоинт через symlink nanoGPT-lab
ckpt_path = 'nanoGPT-lab/model_ckpt.pt'

if os.path.exists(ckpt_path):
    state_dict = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(state_dict)
    print(f"✅ Чекпоинт {ckpt_path} успешно загружен.")
else:
    print(f"⚠️  ВНИМАНИЕ: Файл {ckpt_path} не найден. Работаем с неинициализированными весами.")

✅ Чекпоинт nanoGPT-lab/model_ckpt.pt успешно загружен.


## 2. Иерархия слоев PyTorch

Давайте рекурсивно посмотрим на все модули модели.

In [3]:
def print_structure(module, indent=0):
    for name, child in module.named_children():
        print("  " * indent + f"- {name}: {type(child).__name__}")
        print_structure(child, indent + 1)

print("Структура GPTLanguageModel:")
print_structure(model)

Структура GPTLanguageModel:
- token_embedding_table: Embedding
- position_embedding_table: Embedding
- blocks: Sequential
  - 0: Block
    - sa: MultiHeadAttention
      - heads: ModuleList
        - 0: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
        - 1: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
        - 2: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
        - 3: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
        - 4: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
        - 5: Head
          - key: Linear
          - query: Linear
          - value: Linear
          - dropout: Dropout
      - proj: Linear
      - dropout: Dropout
    - ffwd: FeedFoward
   

## 3. Детальный осмотр первого блока Transformer

Посмотрим на веса слоев в `model.blocks[0]`.

In [4]:
first_block = model.blocks[0]
print(f"Осмотр блока: {first_block}\n")

for name, param in first_block.named_parameters():
    print(f"{name:40} | Shape: {str(list(param.shape)):20} | Mean: {param.mean().item():.4f}")

Осмотр блока: Block(
  (sa): MultiHeadAttention(
    (heads): ModuleList(
      (0-5): 6 x Head(
        (key): Linear(in_features=384, out_features=64, bias=False)
        (query): Linear(in_features=384, out_features=64, bias=False)
        (value): Linear(in_features=384, out_features=64, bias=False)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
    (proj): Linear(in_features=384, out_features=384, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (ffwd): FeedFoward(
    (net): Sequential(
      (0): Linear(in_features=384, out_features=1536, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1536, out_features=384, bias=True)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
)

sa.heads.0.key.weight                    | Shape: [64, 384]            | Mean: 0.0000
sa.heads.0.query.weight                  | Shape: [64, 384]

## 4. Расширенный SVD анализ (Query, Key, Value для всех голов)

Теперь мы проанализируем не только Query, но и Key/Value матрицы для всех голов первого блока. 
Мы ищем 'слабые' матрицы — те, у которых количество значимых сингулярных чисел (выше 10% от максимума) меньше 64 (размера головы).

### 2. Главный инструмент — SVD (Singular Value Decomposition)

Для анализа и сжатия матриц мы используем разложение:
$$W = U \Sigma V^T$$

- $\Sigma = \text{diag}(\sigma_1 \ge \sigma_2 \ge \dots \ge 0)$ — диагональная матрица сингулярных чисел.
- $\sigma_i$ — сингулярные значения, упорядоченные по невозрастанию.

📌 **Что они показывают?**
Каждое сингулярное значение $\sigma_i$ буквально показывает, **сколько «информации» или «энергии»** несет в себе соответствующее ему направление (главная компонента). Чем быстрее убывают значения $\sigma_i$, тем больше матрица «склонна» к сильному сжатию без потери качества.

### Математическое обоснование: Как выбирать ранг $r$?

SVD — это не просто способ разложить матрицу, это инструмент для оптимального сжатия информации. Вот ключевые способы выбора ранга $r$ для аппроксимации $W \approx U_r \Sigma_r V_r^T$:

#### 1. По сохранению энергии (наиболее популярный)
Мы выбираем минимальный ранг $r$, который сохраняет заданный процент «энергии» (дисперсии) матрицы:
$$\frac{\sum_{i=1}^{r} \sigma_i^2}{\sum_{i=1}^{d} \sigma_i^2} \ge 1 - \varepsilon$$

**Типичные значения:**
- $\varepsilon = 0.01 \rightarrow$ сохраняем 99% энергии.
- $\varepsilon = 0.05 \rightarrow$ сохраняем 95% энергии.

📌 *В современных LLM часто оказывается, что для матриц весов достаточно $r = 16\dots128$ при исходном размере 4096+.*

#### 2. Через ошибку аппроксимации
Согласно свойствам SVD, квадрат нормы Фробениуса разности между оригиналом и аппроксимацией равен сумме квадратов отброшенных сингулярных чисел:
$$\|W - W_r\|_F^2 = \sum_{i=r+1}^{d} \sigma_i^2$$
Вы выбираете $r$, при котором ошибка падает ниже допустимого порога.

#### 3. Почему именно SVD? (Теорема Эккарта–Юнга)
Это критически важный момент. Почему мы не используем другие разложения для сжатия? 

> **Теорема Эккарта–Юнга:** Усеченное SVD (Truncated SVD) дает **наилучшую** возможную аппроксимацию ранга $r$ для любой матрицы в смысле нормы Фробениуса и спектральной нормы.

📌 *Это не эмпирическое правило (эвристика) — это строгая математическая гарантия того, что SVD извлекает максимум информации в заданный объем памяти.*

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

def analyze_matrix_comprehensive(W, energy_threshold=0.95, sig_threshold_pct=0.1):
    W_cpu = W.detach().cpu()
    U, S, V = torch.linalg.svd(W_cpu)
    S_np = S.numpy()
    
    # 1. Порог по значению (10% от максимума)
    max_val = S_np[0]
    significant_val = np.sum(S_np > (max_val * sig_threshold_pct))
    
    # 2. Порог по энергии (95% суммы квадратов)
    total_energy = np.sum(S_np**2)
    cumulative_energy = np.cumsum(S_np**2) / total_energy
    significant_energy = np.argmax(cumulative_energy >= energy_threshold) + 1
    
    return S_np, significant_val, significant_energy

block_idx = 0
block = model.blocks[block_idx]
num_heads = len(block.sa.heads)

print(f"--- Расширенный анализ сингулярности (Блок {block_idx}) ---")
print(f"Размер головы: 64")
print(f"{'Матрица':<15} | {'Sig (>10%)':<12} | {'Energy (95%)':<12}")
print("-" * 45)

fig, axes = plt.subplots(num_heads, 3, figsize=(15, 3 * num_heads))

for h in range(num_heads):
    head = block.sa.heads[h]
    for i, (m_name, layer) in enumerate([('Query', head.query), ('Key', head.key), ('Value', head.value)]):
        W = layer.weight.data
        S_np, sig_val, sig_eng = analyze_matrix_comprehensive(W)
        
        full_name = f"H{h} {m_name}"
        print(f"{full_name:<15} | {sig_val:<12} | {sig_eng:<12}")
        
        ax = axes[h, i]
        ax.plot(S_np, label=r'$\sigma_i$')
        # Отрисовка порога 95% энергии
        ax.axvline(x=sig_eng, color='orange', linestyle='--', alpha=0.5, label='95% Energy')
        ax.set_title(f"{full_name}\nEng95: {sig_eng}/64")
        ax.grid(True)
        if h == 0 and i == 0: ax.legend()
        
plt.tight_layout()
plt.show()

print("\n📌 Интерпретация:")
print("1. Если 'Sig (>10%)' = 64, это значит, что сингулярные числа убывают очень медленно (матрица плотная).")
print("2. Если 'Energy (95%)' заметно меньше 64, то матрица допускает эффективное сжатие.")
print("3. В маленьких моделях (как nanoGPT) головы часто имеют полный ранг, так как 'лишних' параметров меньше, чем в моделях на 7B+.")